[View on GitHub](https://github.com/csc-training/sscc-cp2k)

# Proton transfer simulations with CP2K

## Part 2: Normal mode analysis (NMA) of the transition state

The transition state (TS) is a first-order saddle-point on the multi-dimensional potential energy surface (PES). In other words, it is a maximum along the reaction path and a minimum in all other directions. This means that a normal mode analysis (NMA) of the TS should yield exactly one imaginary vibrational mode. This technique is frequently used to verify that a TS was really found.

### Step 2.1: Prepare the input file and submit the calculation

**Start by copying the optimized TS structure and wave function restart file to the current working directory:**

In [ ]:
%%bash
cp ../Part-1-NEB/enol-neb-BAND5-RESTART.wfn .
cp ../Part-1-NEB/enol-neb-pos-Replica_nr_5-1.xyz .

**Inspect the input file `nma.inp` and fix all occurrences of `FIXME` based on that:**

* Run type is `NORMAL_MODES`
* Wavefunction restart file name is `enol-neb-BAND5-RESTART.wfn`
* Coordinate file name is `enol-neb-pos-Replica_nr_5-1.xyz`
* Initial density guess is made based on `RESTART`

Write the input to a file `nma.inp` by running the cell.

In [ ]:
%%writefile nma.inp
&GLOBAL
  PROJECT enol-nma                      ! Project name, gets prepended to all output files
  RUN_TYPE FIXME                        ! Type of calculation
  PRINT_LEVEL LOW                       ! Low verbosity
&END GLOBAL

&FORCE_EVAL                             ! Include DFT and system settings
  &DFT
    RESTART_FILE_NAME FIXME
    BASIS_SET_FILE_NAME BASIS_MOLOPT
    &QS
      EPS_DEFAULT 1.0E-12               ! Sets convergence criteria so that energy should be accurate up to this value
    &END QS
    &POISSON                            ! How to deal with electrostatics
      PERIODIC NONE                     ! System is not periodic
      POISSON_SOLVER WAVELET            ! Efficient solver for non-periodic system
    &END POISSON
    &SCF
      SCF_GUESS FIXME                   ! How to guess initial electron density for SCF cycle
      EPS_SCF 1.0E-6                    ! Energy convergence criteria (atomic units)
      &OT                               ! Use orbital transformation method instead of brute-force diagonalization
        ALGORITHM IRAC                  ! OT algorithm
        MINIMIZER DIIS                  ! Minimization algorithm
        PRECONDITIONER FULL_ALL         ! Preconditioner for the minimizer
      &END OT
      MAX_SCF 20                        ! Maximum number of SCF steps before rebuilding preconditioner
      &OUTER_SCF ON
        EPS_SCF 1.0E-6                  ! Outer loop energy convergence criteria (atomic units)
        MAX_SCF 12                      ! Maximum number of preconditioner rebuilds before terminating
      &END OUTER_SCF
      &PRINT
        &RESTART
          BACKUP_COPIES 0               ! Avoid excessive printing of restart files
        &END RESTART
      &END PRINT
    &END SCF
    &XC
      &XC_FUNCTIONAL PBE                ! Exchange-correlation funtional
      &END XC_FUNCTIONAL
    &END XC
  &END DFT
  &SUBSYS
    &CELL
      PERIODIC NONE                     ! System is not periodic
      ABC 15.0 15.0 15.0                ! Simulation cell dimensions in angstroms
    &END CELL
    &TOPOLOGY                           ! Specify input coordinate format and filename
      COORD_FILE_FORMAT XYZ
      COORD_FILE_NAME FIXME
    &END TOPOLOGY
    &KIND H                             ! Basis and pseudopotential for hydrogen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q1
    &END KIND
    &KIND C                             ! Basis and pseudopotential for carbon
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q4
    &END KIND
    &KIND O                             ! Basis and pseudopotential for oxygen
      BASIS_SET DZVP-MOLOPT-SR-GTH
      POTENTIAL GTH-PBE-q6
    &END KIND
  &END SUBSYS
&END FORCE_EVAL

&VIBRATIONAL_ANALYSIS
  NPROC_REP 10
&END VIBRATIONAL_ANALYSIS

**Next, write the following simple batch job script to a file by running the cell.** We'll again request two full Puhti nodes (40 CPU cores each).

In [ ]:
%%writefile job.sh
#!/bin/bash
#SBATCH --time=00:15:00
#SBATCH --reservation=sscc_thu_large
#SBATCH --partition=large
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=40
#SBATCH --account=project_2006657

module purge
module load intel-oneapi-compilers-classic/2021.6.0
module load intel-oneapi-mpi/2021.6.0
module load cp2k/2024.1

srun cp2k.psmp nma.inp

**Run the calculation as a batch job by executing the cell below.** Note, before running `sbatch`, we unset all Slurm environment variables as we are submitting a batch job from another Slurm job (this notebook that we're currently running).

In [ ]:
!unset ${!SLURM@}; sbatch job.sh

The calculation should only take about 3 minutes. You may monitor the queue using `squeue` command:

In [ ]:
!squeue --me

### Step 2.2: Analysis

**Once the job has completed, inspect the produced `.mol` file:**

In [ ]:
!cat enol-nma-VIBRATIONS-1.mol

* Check the lines under `[FREQ]`. Is there only one imaginary (negative) frequency?
* Scroll to the end of the file. How many vibrational modes are there in total? Does the number make sense?
    * Hint: think about degrees of freedom.

### Bonus: Visualize the imaginary vibrational mode

[Molden](https://docs.csc.fi/apps/molden/) can be used to visualize `.mol` files. However, in a Jupyter Notebook we need to use other tools such as py3Dmol. To this end, create a new `.xyz` file that contains both the atomic and normal mode coordinates:

In [ ]:
%%bash
tail -21 enol-neb-pos-Replica_nr_5-1.xyz > coord.tmp
grep -A20 FR-NORM-COORD enol-nma-VIBRATIONS-1.mol > vib.tmp
paste coord.tmp vib.tmp > vib.xyz

Then, you can visualize the mode using py3Dmol:

In [ ]:
import py3Dmol

with open('vib.xyz', 'r') as f:
    vib = f.read()

view = py3Dmol.view(height=400, width=400)
view.addModel(vib, 'xyz', {'vibrate': {'frames': 10, 'amplitude': 0.5}})
view.setStyle({'sphere': {'scale': 0.30}, 'stick': {'radius': 0.25}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

**[Continue the hands-on with Part 3.](../Part-3-Hybrid/Part-3-Hybrid.ipynb)**